In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
import anndata
import pathlib

In [2]:
bed_dir = pathlib.Path('filtered_bed/')
bed_list = list(bed_dir.glob('*.filtered_enhancer_dmr.bed'))

In [3]:
total_dmr = pd.read_hdf('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5', key='bed')

In [4]:
records = {}
for i, path in enumerate(bed_list):
    data = pd.read_csv(path, header=None, index_col=3, sep='\t')[4]
    records[path.name.split('.')[0]] = data.reindex(total_dmr.index).fillna(0)
    print(i, path)
records = pd.DataFrame(records)

0 filtered_bed/CA3_Efnb2.filtered_enhancer_dmr.bed
1 filtered_bed/D1L-Fstl4_Sipa1l2.filtered_enhancer_dmr.bed
2 filtered_bed/ANP_anp-dg.filtered_enhancer_dmr.bed
3 filtered_bed/D1L-Fstl4_Trps1.filtered_enhancer_dmr.bed
4 filtered_bed/ANP_anp-olf-cnu.filtered_enhancer_dmr.bed
5 filtered_bed/D1L-PAL_Flrt2.filtered_enhancer_dmr.bed
6 filtered_bed/ASC_cortex-olf.filtered_enhancer_dmr.bed
7 filtered_bed/MGE-Pvalb_Cnih3.filtered_enhancer_dmr.bed
8 filtered_bed/ASC_mid.filtered_enhancer_dmr.bed
9 filtered_bed/D1L-PAL_Plcxd3.filtered_enhancer_dmr.bed
10 filtered_bed/ASC_str-hpf.filtered_enhancer_dmr.bed
11 filtered_bed/MGE-Pvalb_Gfra2.filtered_enhancer_dmr.bed
12 filtered_bed/CA1_Ak5.filtered_enhancer_dmr.bed
13 filtered_bed/DG_dg-all.filtered_enhancer_dmr.bed
14 filtered_bed/CA1_Chrm3.filtered_enhancer_dmr.bed
15 filtered_bed/DG-po_Bcl11a.filtered_enhancer_dmr.bed
16 filtered_bed/CA1_Kif26a.filtered_enhancer_dmr.bed
17 filtered_bed/Foxp2_Dchs2.filtered_enhancer_dmr.bed
18 filtered_bed/CA1_Lin

148 filtered_bed/OLF-Exc_Cdh9.filtered_enhancer_dmr.bed
149 filtered_bed/OLF-Exc_Cux2.filtered_enhancer_dmr.bed
150 filtered_bed/OLF-Exc_Lrrtm3.filtered_enhancer_dmr.bed
151 filtered_bed/OLF-Exc_Pld5.filtered_enhancer_dmr.bed
152 filtered_bed/OLF-Exc_Rmst.filtered_enhancer_dmr.bed
153 filtered_bed/OLF-Exc_Sgcd.filtered_enhancer_dmr.bed
154 filtered_bed/OLF-Exc_Unc13c.filtered_enhancer_dmr.bed
155 filtered_bed/OLF_Kcnd3.filtered_enhancer_dmr.bed
156 filtered_bed/OLF_Mapk10.filtered_enhancer_dmr.bed
157 filtered_bed/OLF_Pag1.filtered_enhancer_dmr.bed
158 filtered_bed/OLF_Trpc4.filtered_enhancer_dmr.bed
159 filtered_bed/OLF_Xkr6.filtered_enhancer_dmr.bed
160 filtered_bed/OPC_opc-large.filtered_enhancer_dmr.bed


In [9]:
records[records < 0.1] = 0

In [10]:
adata = anndata.AnnData(X=csc_matrix(records.values),
                        obs=total_dmr,
                        var=pd.DataFrame([], index=records.keys()))

In [11]:
adata.write_h5ad('REPTILE_scores.subtype.h5ad')

... storing '#chr' as categorical


In [2]:
adata = anndata.read_h5ad('REPTILE_scores.subtype.h5ad')

In [9]:
dmr_total_hits = (adata.X > 0).sum(axis=1).A1
dmr_total_hits = pd.Series(dmr_total_hits)